# 0. Installation (one time job)

In [1]:
#!pip install scikit-learn==0.23.1

In [2]:
# xlrd no longer support xlsx - https://stackoverflow.com/questions/65254535/xlrd-biffh-xlrderror-excel-xlsx-file-not-supported
!pip install openpyxl

# 1. Import Library

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from TextPreprocessing import text_preprocessing

# 2. Check Data

In [4]:
# Read data
pd.set_option('display.max_columns', None)

# data = pd.read_excel('./ASD FAQ KB v001.xlsx', sheet_name='FAQ')
data = pd.read_excel('./ASD FAQ KB v001.xlsx', sheet_name='FAQ', engine='openpyxl')

data.head()

,sn,Question,Long_Answer,Short_Answer,Source,Remarks
0,1,What are Autism Spectrum Disorders (ASD)?,ASD refers to a wide spectrum of neurodevelopm...,NaN,http://birchtreecenter.org/learn/autism,NaN
1,2,How common is autism?,According to a 2020 report commissioned by the...,NaN,http://birchtreecenter.org/learn/autism,NaN
2,3,What causes autism? Can it be cured?,The causes of this complex disorder remain unc...,NaN,http://birchtreecenter.org/learn/autism,NaN
3,4,Why doesn’t intervention center refer to its s...,Our students are children or youth who are cha...,NaN,http://birchtreecenter.org/learn/autism,NaN
4,5,What are the types of Autism Spectrum Disorders?,Autistic Disorder; Asperger Syndrome; Pervasiv...,NaN,http://dhss.alaska.gov/dph/wcfh/Pages/autism/s...,NaN


In [5]:
print(data.shape)

(226, 6)


# 3. Data Preprocessing

In [6]:
# Select long_answer from the data
long_answer = data.get('Long_Answer')

# Preprocess the long_answer
long_answer = long_answer.map(lambda x: ' '.join(text_preprocessing(x)))

# Vectorize the answers (one-hot)
sparse_vectorizer = CountVectorizer(strip_accents = 'unicode')
sparse_vectors = sparse_vectorizer.fit_transform(long_answer)

# (226, 2753)
print(sparse_vectors.shape)

(226, 2753)


# 4. Build Topic Model using LDA

In [7]:
# Your super power to define number of topics
n_topics = 4

# Run LDA to generate topics/clusters
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=1000,
                                learning_method='online',
                                random_state=0)

lda.fit(sparse_vectors)

LatentDirichletAllocation(learning_method='online', max_iter=1000,
                          n_components=4, random_state=0)

# 5. Display the resulting topics/clusters of ASD FAQ's Long_Answer field

In [8]:
# Print the top-n key words
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [9]:
# Show the first n_top_words key words
n_top_words = 10
feature_names = sparse_vectorizer.get_feature_names()

for i, topic in enumerate(lda.components_):
    print('Topic {num}'.format(num=i+1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic 1
autism disorder asd child may spectrum people behavior social cause

Topic 2
child autism may treatment intervention diagnosis early help parent therapy

Topic 3
institute national autism tel information health disorder fax behavior md

Topic 4
ability assessment behaviour concern specific characteristic academic ot memory diet



In [10]:
print("1st document(long FAQ answer) belongs to Topic",lda.transform(sparse_vectors[0]).argmax()+1)

1st document(long FAQ answer) belongs to Topic 1


In [11]:
# Display topics of first 20 long FAQ answers:
for i in range(0, 20):
    print("Document (long FAQ answer)", i+1, "belongs to Topic", lda.transform(sparse_vectors[i]).argmax()+1)

Document (long FAQ answer) 1 belongs to Topic 1
Document (long FAQ answer) 2 belongs to Topic 1
Document (long FAQ answer) 3 belongs to Topic 1
Document (long FAQ answer) 4 belongs to Topic 1
Document (long FAQ answer) 5 belongs to Topic 1
Document (long FAQ answer) 6 belongs to Topic 1
Document (long FAQ answer) 7 belongs to Topic 3
Document (long FAQ answer) 8 belongs to Topic 1
Document (long FAQ answer) 9 belongs to Topic 1
Document (long FAQ answer) 10 belongs to Topic 1
Document (long FAQ answer) 11 belongs to Topic 1
Document (long FAQ answer) 12 belongs to Topic 1
Document (long FAQ answer) 13 belongs to Topic 1
Document (long FAQ answer) 14 belongs to Topic 1
Document (long FAQ answer) 15 belongs to Topic 1
Document (long FAQ answer) 16 belongs to Topic 2
Document (long FAQ answer) 17 belongs to Topic 2
Document (long FAQ answer) 18 belongs to Topic 2
Document (long FAQ answer) 19 belongs to Topic 2
Document (long FAQ answer) 20 belongs to Topic 2


# 6. Interpret the identified topics (using top 10 words)

### Topic 1 is about: 
Definition of autism (ASD), its frequency in children and diverse populations, and related social behaviors and causes.

### Topic 2 is about: 
Diagnosis, early intervention, and treatment methods for children with autism, to provide assistance and support for parents.

### Topic 3 is about: 
According to the information provided by the National Autism Research Institute, including health and behavioral disorders, they provide services through telephone (tel) and fax.

### Topic 4 is about: 
Ability assessment, behavioral issues, specific characteristics, academic performance, occupational therapy, memory and diet, and other related topics.


# 7. Food for thought: What about the clusters/topics of "Questons"?

---
`The end is called the new start.` --- ISS : **I** **S**(elf) **S**(tudy)